In [1]:
%reload_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import torch
import torchvision
import torchprof as tp
import time

model = torchvision.models.alexnet(pretrained=False).cuda()
x = torch.rand([1, 3, 224, 224]).cuda()


# Regions

In [3]:
with tp.profile(model, use_cuda=True) as prof:
    with tp.region("Arbitrary region name"):
        _ = model(x)
    with tp.region("Another region"):
        time.sleep(1)

prof.display()

Adding traces:   0%|          | 0/24 [00:00<?, ?it/s]

Adding traces: 100%|██████████| 24/24 [00:00<00:00, 87154.37it/s]

Removing traces:   0%|          | 0/24 [00:00<?, ?it/s]

Removing traces: 100%|██████████| 24/24 [00:00<00:00, 150919.48it/s]


Make Events:   0%|          | 0/262 [00:00<?, ?it/s]

...


Aggregate:   0%|          | 0/184 [00:00<?, ?it/s]

Aggregate: 100%|██████████| 184/184 [00:00<00:00, 98942.56it/s]


Make rows: 0it [00:00, ?it/s]

Make rows: 28it [00:00, 2998.53it/s]



CPU=1.0s, CUDA=1.0s
+---------------------------+------------+-------------+-------------+-------------+-----+
| Node                      |   Self CPU |         CPU |   Self CUDA |        CUDA |   # |
|---------------------------+------------+-------------+-------------+-------------+-----|
| <torchprof>               |            | 1.0s (100%) |             | 1.0s (100%) |   1 |
|  ├──Another region        | 1.0s (98%) |  1.0s (98%) |  1.0s (98%) |  1.0s (98%) |   1 |
|  └──Arbitrary region name |            | 16.5ms (2%) |             | 16.5ms (2%) |   1 |
|    └──AlexNet             |            | 16.4ms (2%) |             | 16.5ms (2%) |   1 |
+---------------------------+------------+-------------+-------------+-------------+-----+


# Functions

In [4]:
@tp.func()
def foo():
    _ = model(x)

with tp.profile(model) as prof:
    foo()

prof.display()    

Adding traces:   0%|          | 0/24 [00:00<?, ?it/s]

Adding traces: 100%|██████████| 24/24 [00:00<00:00, 47393.27it/s]


Removing traces:   0%|          | 0/24 [00:00<?, ?it/s]

Removing traces: 100%|██████████| 24/24 [00:00<00:00, 156796.41it/s]


Make Events:   0%|          | 0/257 [00:00<?, ?it/s]

...


Aggregate:   0%|          | 0/182 [00:00<?, ?it/s]

Aggregate: 100%|██████████| 182/182 [00:00<00:00, 117224.10it/s]


Make rows: 0it [00:00, ?it/s]

Make rows: 27it [00:00, 1329.81it/s]



CPU=13.7ms, CUDA=0.0µs
+--------------------+--------------+---------------+-------------+--------+-----+
| Node               |     Self CPU |           CPU |   Self CUDA |   CUDA |   # |
|--------------------+--------------+---------------+-------------+--------+-----|
| <torchprof>        | 281.0µs (2%) | 13.7ms (100%) |             |        |   1 |
|  └──foo()          |              |  13.4ms (98%) |             |        |   1 |
|    └──AlexNet      | 211.0µs (2%) |  13.2ms (97%) |             |        |   1 |
|      ├──features   | 651.3µs (5%) |   5.3ms (39%) |             |        |   1 |
|      │ ├──8        | 720.0µs (5%) |  803.5µs (6%) |             |        |   1 |
|      │ ├──0        | 252.0µs (2%) |  713.1µs (5%) |             |        |   1 |
|      │ ├──5        |              |  224.5µs (2%) |             |        |   1 |
|      │ ├──3        | 940.8µs (7%) |    1.0ms (8%) |             |        |   1 |
|      │ ├──6        | 551.9µs (4%) |  655.6µs (5%) |         

# Iterators

In [5]:
foo = tp.iter()(time.sleep(0.5) for _ in range(3))
foo2 = tp.iter("bar")(time.sleep(0.1) for _ in range(2))

with tp.profile(model) as prof:
    _ = list(foo)
    with tp.region("renamed"):
        _ = list(foo2)

prof.display()    

Adding traces:   0%|          | 0/24 [00:00<?, ?it/s]

Adding traces: 100%|██████████| 24/24 [00:00<00:00, 114781.41it/s]

Removing traces:   0%|          | 0/24 [00:00<?, ?it/s]

Removing traces: 100%|██████████| 24/24 [00:00<00:00, 142582.57it/s]


Make Events:   0%|          | 0/45 [00:00<?, ?it/s]

...


Aggregate:   0%|          | 0/17 [00:00<?, ?it/s]

Aggregate: 100%|██████████| 17/17 [00:00<00:00, 69700.07it/s]


Make rows: 0it [00:00, ?it/s]

Make rows: 5it [00:00, 828.13it/s]



CPU=1.7s, CUDA=0.0µs
+-----------------+----------------------+----------------------+-------------+--------+-----+
| Node            |             Self CPU |                  CPU |   Self CUDA |   CUDA |   # |
|-----------------+----------------------+----------------------+-------------+--------+-----|
| <torchprof>     |                      |          1.7s (100%) |             |        |   1 |
|  ├──next(foo)   |   1.5s/375.8ms (88%) |   1.5s/375.9ms (88%) |             |        |   4 |
|  └──renamed     |                      |        201.7ms (12%) |             |        |   1 |
|    └──next(bar) | 201.4ms/67.1ms (12%) | 201.4ms/67.1ms (12%) |             |        |   3 |
+-----------------+----------------------+----------------------+-------------+--------+-----+


# Global Settings

In [6]:
with tp.global_settings(enabled=False):
    with tp.profile(model) as prof:
        with tp.region("foo"):
            time.sleep(1)

prof.display()

Adding traces:   0%|          | 0/24 [00:00<?, ?it/s]

Adding traces: 100%|██████████| 24/24 [00:00<00:00, 92013.98it/s]

Removing traces:   0%|          | 0/24 [00:00<?, ?it/s]

Removing traces: 100%|██████████| 24/24 [00:00<00:00, 129055.51it/s]


Make Events: 0it [00:00, ?it/s]

...


Group: 0it [00:00, ?it/s]

Group: 0it [00:00, ?it/s]


Aggregate: 0it [00:00, ?it/s]

Aggregate: 0it [00:00, ?it/s]



CPU=0.0µs, CUDA=0.0µs


In [7]:
# For use with Nsight systems etc.
with tp.global_settings(nvtx=True):
    with tp.region("foo"):
        time.sleep(1)
